In [12]:
import requests
import re
import time
import pickle
import numpy as np

In [41]:
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

In [11]:
HEADERS = {
    'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'
}

In [7]:
URL = r'https://www.bbcgoodfood.com/recipes/next-level-chocolate-fondant'

In [13]:
page = requests.get(URL, headers=HEADERS)

In [14]:
page.content

b'<!doctype html>\n<!--[if IE 7]>    <html class="no-js lt-ie9 lt-ie8 no-touch" lang="en"> <![endif]-->\n<!--[if IE 8]>    <html class="no-js lt-ie9 no-touch" lang="en"> <![endif]-->\n<!--[if gt IE 8]> <html class="no-js gt-ie-8 no-touch" lang="en"> <![endif]-->\n<!--[if !IE]><!--><html class="no-js no-touch" lang="en"> <!--<![endif]-->\n  <head>\n    <meta charset="utf-8">\n    <title>Next level chocolate fondant recipe | BBC Good Food</title>\n    <!--[if IE]><![endif]-->\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n<link rel="apple-touch-icon-precomposed" href="/sites/all/themes/bbcw_goodfood/images/good-food-icon/good-food-icon-72x72_2017.png" sizes="72x72" />\n<link rel="shortcut icon" href="https://www.bbcgoodfood.com/sites/all/themes/bbcw_goodfood/images/GF-favicon-2015-16x16.ico" type="image/vnd.microsoft.icon" />\n<meta name="msApplication-PackageFamilyName" content="BBCWorldwideLtd.BBCGoodFood_g2s5c4trewyqj" />\n<meta name="msApplication-ID" content

In [30]:
ingredients_re = re.compile('ingredients:\[\"(.*?)\"\]')

In [44]:
[cleanhtml(ingred) for ingred in ingredients_re.findall(str(page.content))[0].split("\",\"")]

['butter',
 'cocoa powder',
 'caramel sauce',
 'dark chocolate',
 'coffee',
 'egg',
 'golden caster sugar',
 'plain flour',
 'pudding basins']

In [45]:
recipe_re = re.compile('<li class="method__item" itemprop="recipeInstructions"><p>(.*?)<\/p><\/li>')

In [46]:
[cleanhtml(step) for step in recipe_re.findall(str(page.content))]

['Use 50g of the butter to brush the insides of the pudding basins. Put them on a baking tray and transfer to the fridge to chill until the butter sets. Brush over another layer of the butter, then dust generously with the cocoa. Return to the fridge to set. Mix the caramel sauce with a good pinch of sea salt flakes, then chill in the fridge until ready to use.&nbsp;',
 'Tip the remaining butter, the chocolate and coffee powder into a heatproof bowl. Set the bowl over a pan of gently simmering water, ensuring the bottom of the bowl doesn&rsquo;t touch the water. Stir the mixture until smooth, adding another small pinch of sea salt. Tip the whole eggs, egg yolks and sugar into another bowl and beat together using an electric whisk until very pale &ndash; the whisk should leave a trail in the mixture when lifted. Sift in the flour, then gently fold in the melted chocolate mixture.',
 'Spoon the fondant mixture into the prepared basins until just about halfway full, then transfer to the f

In [47]:
ingredients_ammounts_re = re.compile('<li class="ingredients-list__item" itemprop="ingredients" content="(.*?)"')

In [48]:
[cleanhtml(i) for i in ingredients_ammounts_re.findall(str(page.content))]

['200g butter, melted',
 '3 tbsp cocoa powder, for dusting',
 '50g caramel sauce',
 '200g dark chocolate, chopped',
 '4 tbsp very strong coffee powder (espresso is ideal)',
 '6 eggs, three whole, plus the yolks of 3 (freeze the leftover egg whites for another recipe)',
 '75g golden caster sugar',
 '50g plain flour',
 '6 small metal pudding basins']

In [52]:
title_re = re.compile('<title>(.*?)<\/title>')
recipe_re = re.compile('<li class="method__item" itemprop="recipeInstructions"><p>(.*?)<\/p><\/li>')
ingredients_re = re.compile('ingredients:\[\"(.*?)\"\]')
ingredientsAmmounts_re = re.compile('<li class="ingredients-list__item" itemprop="ingredients" content="(.*?)"')


def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

def getRecipe(URL, headers=None):
    if headers == None:
        headers = {'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'}
    
    page = requests.get(URL, headers=HEADERS)
    pageText = str(page.content)
    title = title_re.findall(pageText)[0]
    steps = [cleanhtml(i) for i in recipe_re.findall(pageText)]
    ingredients = [cleanhtml(i) for i in ingredients_re.findall(pageText)]
    ammounts = [cleanhtml(i) for i in ingredientsAmmounts_re.findall(pageText)]
    return [title, URL, ingredients, ammounts, steps]

getRecipe(URL)

['Next level chocolate fondant recipe | BBC Good Food',
 'https://www.bbcgoodfood.com/recipes/next-level-chocolate-fondant',
 ['butter","cocoa powder","caramel sauce","dark chocolate","coffee","egg","golden caster sugar","plain flour","pudding basins'],
 ['200g butter, melted',
  '3 tbsp cocoa powder, for dusting',
  '50g caramel sauce',
  '200g dark chocolate, chopped',
  '4 tbsp very strong coffee powder (espresso is ideal)',
  '6 eggs, three whole, plus the yolks of 3 (freeze the leftover egg whites for another recipe)',
  '75g golden caster sugar',
  '50g plain flour',
  '6 small metal pudding basins'],
 ['Use 50g of the butter to brush the insides of the pudding basins. Put them on a baking tray and transfer to the fridge to chill until the butter sets. Brush over another layer of the butter, then dust generously with the cocoa. Return to the fridge to set. Mix the caramel sauce with a good pinch of sea salt flakes, then chill in the fridge until ready to use.&nbsp;',
  'Tip the r

In [65]:
recipes_All = "https://www.bbcgoodfood.com/search_api_ajax/search/recipes?sort=field_bbcgf_total_time_seconds&order=asc&page="
base_url = "https://www.bbcgoodfood.com/recipes/"

In [66]:
url_extract_re = re.compile('teaser-item__image\"> <a href=\"\/recipes\/(.*?)\">')

In [95]:
noErr = True
i=0
recipes = []

while noErr:
    if i == 0:
        page = requests.get(recipes_All, headers=HEADERS)
    else:
        page = requests.get(recipes_All + str(i), headers=HEADERS)
    pageText = str(page.content)
    #print(pageText)
    raws = url_extract_re.findall(pageText)
    urls = [base_url + u for u in raws]
    for url in urls:
        rec = getRecipe(url)
        recipes.append(rec)
    if page.status_code != 200:
        print(page)
        noErr = False
    print(len(recipes))
    i = i + 1
    
    time.sleep(0)

KeyboardInterrupt: 

In [92]:
len(recipes)

12706

In [5]:
recipes_bak = recipes

In [96]:
recipes = recipes_bak

In [94]:
[print(each[0]) for each in recipes]

Salted caramel & macadamia pralines recipe | BBC Good Food
Onion rings recipe | BBC Good Food
Roast duck & griddled plum salad recipe | BBC Good Food
Spiced duck & date tagine recipe | BBC Good Food
Greek courgetti salad recipe | BBC Good Food
Courgette conserve recipe | BBC Good Food
Baileys custard tart recipe | BBC Good Food
Sourdough starter kit recipe | BBC Good Food
Whisky & pink peppercorn marmalade kit recipe | BBC Good Food
Gravadlax kit recipe | BBC Good Food
Bespoke martini kit recipe | BBC Good Food
Bean & feta spread with Greek salad salsa & oatcakes recipe | BBC Good Food
Berry yogurt pots recipe | BBC Good Food
Avocado & strawberry ices recipe | BBC Good Food
Iced grapes with cheddar cubes & celery recipe | BBC Good Food
Chocolate brownie trifle recipe | BBC Good Food
Herb-scented slow-roasted rib of beef recipe | BBC Good Food
Insalata di fave e pecorino (Fresh broad bean & pecorino cheese salad) recipe | BBC Good Food
Spiced almonds recipe | BBC Good Food
Apple crunch 

Cajun turkey wraps with sweetcorn salsa recipe | BBC Good Food
Fusilli with glorious green spinach sauce recipe | BBC Good Food
The ultimate makeover: Chocolate mousse recipe | BBC Good Food
Smoked mackerel with herb & beet couscous recipe | BBC Good Food
Chicory & hummus bites recipe | BBC Good Food
Quick chilli with creamy chive crushed potatoes recipe | BBC Good Food
Kung po prawns recipe | BBC Good Food
Thai chicken skewers recipe | BBC Good Food
Crunchy fruit & nut cereal recipe | BBC Good Food
Crunchy detox salad recipe | BBC Good Food
Smoked salmon spaghetti with chilli & lemon recipe | BBC Good Food
Couscous salad recipe | BBC Good Food
Peach & raspberry sunsets recipe | BBC Good Food
Red chicory, pear & hazelnut salad recipe | BBC Good Food
Spiced chicken & cauliflower pilaf recipe | BBC Good Food
Chocolate berry cups recipe | BBC Good Food
Swiss cheese schnitzel recipe | BBC Good Food
Hummus-crusted lamb with lentil salad recipe | BBC Good Food
Asian-style tofu & cucumber noo

Simple chocolate button egg recipe | BBC Good Food
Sweetcorn & smoked haddock chowder recipe | BBC Good Food
Aubergine & black bean stir-fry recipe | BBC Good Food
All-in-one chicken traybake recipe | BBC Good Food
Chicken masala skewers recipe | BBC Good Food
Coffee cream & walnut cupcakes recipe | BBC Good Food
Must-make moussaka recipe | BBC Good Food
Help-yourself tuna rice salad recipe | BBC Good Food
Curried chickpeas recipe | BBC Good Food
Gujarati cabbage with coconut & potato recipe | BBC Good Food
Egg mayo & watercress rolls recipe | BBC Good Food
Creamy seafood stew recipe | BBC Good Food
Lamb chops with red pepper & mint salsa recipe | BBC Good Food
Skewered sardines with tartare dressing recipe | BBC Good Food
Black pudding potato cakes with fried eggs & tomato chutney recipe | BBC Good Food
Mussels with tomatoes & chilli recipe | BBC Good Food
St David\'s leek & chicken hotpot recipe | BBC Good Food
Pork with sweet & sour onion sauce recipe | BBC Good Food
Pasta with praw

Cheesecake bombe with summer fruits recipe | BBC Good Food
Spicy lamb patties recipe | BBC Good Food
Cheese & Marmite scones recipe | BBC Good Food
Warm stilton salad recipe | BBC Good Food
Frying-pan sausage hotpot recipe | BBC Good Food
Spiced two-bean salad recipe | BBC Good Food
Mexican chilli bean tortilla recipe | BBC Good Food
Panfried fish with fresh beetroot salsa recipe | BBC Good Food
Dill & poppy seed soda bread recipe | BBC Good Food
Pears & prunes in mulled (well, sort of) wine recipe | BBC Good Food
Miniature omelettes with ricotta recipe | BBC Good Food
Cranberry Christmas rockies recipe | BBC Good Food
Roast red mullet with tarragon & pancetta recipe | BBC Good Food
Peppered plums with almond ice recipe | BBC Good Food
Blackberry & clotted cream shortcake recipe | BBC Good Food
Chunky pear & apricot flappers recipe | BBC Good Food
Chicken breasts with rosemary, pine nuts & verjuice recipe | BBC Good Food
Blackcurrant & mint sorbet recipe | BBC Good Food
Herbed potato s

Oriental duck salad recipe | BBC Good Food
Cr\xc3\xa8me caramel recipe | BBC Good Food
Jerk chicken kebabs with mango salsa recipe | BBC Good Food
Chinese-style pork fillet with fried rice recipe | BBC Good Food
Pumpkin, halloumi & chilli omelette recipe | BBC Good Food
Warm roasted squash and Puy lentil salad recipe | BBC Good Food
Tea-smoked salmon with herb mayonnaise recipe | BBC Good Food
Chilli pepper pumpkin with Asian veg recipe | BBC Good Food
Sambar recipe | BBC Good Food
Mexican steak with homemade refried beans recipe | BBC Good Food
Make-ahead Chinese pork recipe | BBC Good Food
Fruit-filled choux buns with caramel sauce recipe | BBC Good Food
Marinated new potatoes recipe | BBC Good Food
Welsh goat\'s cheese & leek tart recipe | BBC Good Food
Grilled steak topped with ceps recipe | BBC Good Food
Mushroom & olive tatins recipe | BBC Good Food
Creamy veggie risotto recipe | BBC Good Food
Cherry & raspberry gratin recipe | BBC Good Food
Creamy pesto chicken with roasted toma

Creamy tarragon chicken bake recipe | BBC Good Food
Mumbai potato wraps with minted yogurt relish recipe | BBC Good Food
New potato & smoked haddock crush recipe | BBC Good Food
Sugar-dusted wedding cookies recipe | BBC Good Food
Whisky cream recipe | BBC Good Food
Chinese braised red cabbage recipe | BBC Good Food
Linguine & pesto frittata recipe | BBC Good Food
Haddock in tomato basil sauce recipe | BBC Good Food
Whole roast bream with potatoes & olives recipe | BBC Good Food
Chicken salad with crisp bacon recipe | BBC Good Food
Butter bean & tomato stew recipe | BBC Good Food
Herrings rolled with mustard & pancetta recipe | BBC Good Food
Crispy fish & chips with mushy peas recipe | BBC Good Food
Coconut-ice marshmallows recipe | BBC Good Food
All-the-trimmings roast potatoes recipe | BBC Good Food
Thai roast chicken with mango & apple salad recipe | BBC Good Food
Pumpkin & bean spaghetti recipe | BBC Good Food
Boxty with bacon, eggs & tomatoes recipe | BBC Good Food
Oregano & lemon 

Oven-roasted chips recipe | BBC Good Food
Warm chickpea, fennel & pepper salad recipe | BBC Good Food
Fish & finger pie recipe | BBC Good Food
Luscious lemon pudding with summer berries recipe | BBC Good Food
Moroccan-style chicken with carrot & orange salad recipe | BBC Good Food
Tomato tart recipe | BBC Good Food
All-in-one roast chicken & veg recipe | BBC Good Food
Roast apples with cinnamon sugar recipe | BBC Good Food
Sweet & sour aubergines recipe | BBC Good Food
Cherry mallow traybake recipe | BBC Good Food
Gingerbread man party cake recipe | BBC Good Food
Griddled aubergines with yogurt & mint recipe | BBC Good Food
Goat\'s cheese, potato & onion tart recipe | BBC Good Food
Roasted new potatoes with chilli cr\xc3\xa8me fra\xc3\xaeche recipe | BBC Good Food
Wild mushroom risotto recipe | BBC Good Food
Noisette potatoes recipe | BBC Good Food
Shepherd\'s pie potatoes recipe | BBC Good Food
Cinnamon pecan sticky buns recipe | BBC Good Food
Chocolate fondant recipe | BBC Good Food


Madeira loaf cake recipe | BBC Good Food
Sour cream rhubarb squares recipe | BBC Good Food
One-pan baked chicken with squash, sage & walnuts recipe | BBC Good Food
Beet & caramelised onion tart recipe | BBC Good Food
Simple roast chicken recipe | BBC Good Food
Marmalade bar recipe | BBC Good Food
Spatchcock barbecue chicken recipe | BBC Good Food
Sweet potato & chorizo quesadillas recipe | BBC Good Food
Minced beef & sweet potato stew recipe | BBC Good Food
Pea, mint & goat\xe2\x80\x99s cheese quiche recipe | BBC Good Food
Tangy beetroot & bean salad recipe | BBC Good Food
Roasted peppers with tomatoes & anchovies recipe | BBC Good Food
Crispy neeps \'n\' tatties cake recipe | BBC Good Food
Chargrilled artichokes with lemon recipe | BBC Good Food
Cheesy celeriac, potato & bacon pie recipe | BBC Good Food
Polenta & pancetta stuffing recipe | BBC Good Food
Cabbage & pork empanadas recipe | BBC Good Food
Caramel panettone pud recipe | BBC Good Food
Wholemeal spinach & potato pies recipe |

Slow cooker spiced root & lentil casserole recipe | BBC Good Food
Malt chocolate cheesecake recipe | BBC Good Food
Onion bacon fougasse recipe | BBC Good Food
Wild venison, field mushroom & ale pudding recipe | BBC Good Food
The ultimate makeover: Coronation chicken recipe | BBC Good Food
Chocolate & chestnut truffle torte recipe | BBC Good Food
Spanish beans with chicken & chorizo recipe | BBC Good Food
Lemon panna cotta with blackberries & honey madeleines recipe | BBC Good Food
Slow-roasted pork with fennel & black pepper recipe | BBC Good Food
BBQ beef short ribs with green apple salsa & fries recipe | BBC Good Food
Brined roast turkey crown & confit legs recipe | BBC Good Food
Slow cooker shepherd\'s pie recipe | BBC Good Food
Slow cooker meatballs recipe | BBC Good Food
Orange berry wedding cake recipe | BBC Good Food
Vanilla panna cotta with caramelised orange recipe | BBC Good Food
Slow-roast shoulder of pork recipe | BBC Good Food
Slow cooker lamb curry recipe | BBC Good Food


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [104]:
pickle.dump(recipes, open("recip.pik","wb"))

In [2]:
recipes  = pickle.load(open("recip.pik","rb"))

In [3]:
print(recipes[0][2])

['macadamia nut","sugar","brown sugar","whole milk","vanilla extract","butter']


In [4]:
all_ing = dict()
for each in recipes:
    for item in each[2]:
        items = item.split('","')
        for a in items:
            if a not in all_ing.keys():
                all_ing[a] = 1
            else:
                all_ing[a] = all_ing[a] + 1

In [5]:
all_ing

{'macadamia nut': 21,
 'sugar': 425,
 'brown sugar': 301,
 'whole milk': 121,
 'vanilla extract': 681,
 'butter': 3385,
 'onion': 2147,
 'groundnut oil': 81,
 'self-raising flour': 700,
 'sparkling water': 36,
 'duck leg': 18,
 'chicken stock': 576,
 'pomegranate juice': 19,
 'cinnamon stick': 189,
 'ground cumin': 306,
 'ground cinnamon': 295,
 'bulgur wheat': 45,
 'plum': 68,
 'olive oil': 3623,
 'green bean': 188,
 'pomegranate': 68,
 'red onion': 913,
 'flaked almond': 183,
 'feta cheese': 155,
 'flat-leaf parsley': 234,
 'mint': 682,
 'pomegranate molasses': 44,
 'extra virgin olive oil': 273,
 'preserved lemon': 29,
 'lemon': 2433,
 'clear honey': 343,
 'sunflower oil': 753,
 'garlic clove': 2976,
 'red chilli': 746,
 'ginger': 877,
 'cumin': 167,
 'coriander': 1393,
 'cinnamon': 365,
 'ground ginger': 124,
 'date': 75,
 'passata': 145,
 'blanched almond': 48,
 'couscous': 144,
 'cucumber': 411,
 'feta': 135,
 'olive': 121,
 'courgetti': 3,
 'courgette': 343,
 'preserving sugar':

In [23]:
double = [[key, all_ing[key]] for key in all_ing.keys()]

In [33]:
list1 = [a[0] for a in double]
list2 = [a[1] for a in double]
b, a = zip(*sorted(zip(list2,list1), reverse=True))

In [34]:
a

('olive oil',
 'butter',
 'garlic clove',
 'lemon',
 'egg',
 'onion',
 'plain flour',
 'coriander',
 'golden caster sugar',
 'milk',
 'caster sugar',
 'double cream',
 'lime',
 'icing sugar',
 'parsley',
 'spring onion',
 'red onion',
 'carrot',
 'ginger',
 'vegetable oil',
 'sunflower oil',
 'red chilli',
 'orange',
 'potato',
 'parmesan',
 'self-raising flour',
 'tomato',
 'mint',
 'vanilla extract',
 'rapeseed oil',
 'baking powder',
 'chicken stock',
 'thyme',
 'vegetable stock',
 'cr\\xc3\\xa8me fra\\xc3\\xaeche',
 'soy sauce',
 'bay leaf',
 'red pepper',
 'shallot',
 'unsalted butter',
 'sugar',
 'dark chocolate',
 'cucumber',
 'honey',
 'cherry tomato',
 'lemon juice',
 'tomato pur\\xc3\\xa9e',
 'egg yolk',
 'large egg',
 'cinnamon',
 'rosemary',
 'avocado',
 'oil',
 'red wine vinegar',
 'white wine vinegar',
 'Dijon mustard',
 'natural yogurt',
 'breadcrumb',
 'courgette',
 'clear honey',
 'basil',
 'nutmeg',
 'chicken breast',
 'cornflour',
 'chicken',
 'apple',
 'leek',
 'dil

In [35]:
b

(3623,
 3385,
 2976,
 2433,
 2330,
 2147,
 1715,
 1393,
 1358,
 1185,
 1162,
 1103,
 1026,
 1000,
 988,
 935,
 913,
 892,
 877,
 850,
 753,
 746,
 730,
 720,
 713,
 700,
 689,
 682,
 681,
 617,
 578,
 576,
 568,
 526,
 520,
 512,
 512,
 501,
 456,
 427,
 425,
 413,
 411,
 404,
 404,
 399,
 376,
 372,
 368,
 365,
 362,
 361,
 357,
 350,
 349,
 349,
 344,
 344,
 343,
 343,
 343,
 339,
 332,
 329,
 328,
 321,
 317,
 316,
 315,
 307,
 306,
 304,
 303,
 302,
 301,
 301,
 297,
 295,
 295,
 295,
 294,
 290,
 289,
 289,
 273,
 272,
 272,
 271,
 269,
 268,
 262,
 261,
 261,
 258,
 257,
 254,
 247,
 245,
 245,
 244,
 241,
 239,
 238,
 234,
 234,
 233,
 233,
 233,
 230,
 226,
 225,
 222,
 221,
 221,
 219,
 215,
 215,
 214,
 209,
 208,
 206,
 205,
 204,
 202,
 202,
 201,
 198,
 198,
 197,
 196,
 192,
 192,
 191,
 190,
 189,
 188,
 188,
 186,
 184,
 184,
 183,
 179,
 179,
 176,
 172,
 169,
 169,
 168,
 168,
 168,
 167,
 167,
 165,
 165,
 160,
 160,
 159,
 158,
 157,
 156,
 155,
 155,
 154,
 154,
 1

In [44]:
recipes[0]

['Salted caramel & macadamia pralines recipe | BBC Good Food',
 'https://www.bbcgoodfood.com/recipes/salted-caramel-macadamia-pralines',
 ['macadamia nut","sugar","brown sugar","whole milk","vanilla extract","butter'],
 ['300g macadamia nut, lightly toasted in a pan or low oven',
  '350g granulated white sugar',
  '175g light brown sugar',
  '125ml whole milk',
  '1 tsp vanilla extract',
  '85g butter'],
 ['First lay a big sheet of baking parchment over your work surface for later. Now mix all the ingredients, with 1 tsp sea salt, in a large saucepan. Cook over a low heat, stirring occasionally, until the sugar has melted, then turn up the heat and bring to the boil.',
  'When boiling, add a sugar thermometer and stir continuously until it reaches hard-ball stage (see tip, left). Remove the pan from the heat, keep stirring for 30 secs then, working quickly using a wooden spoon, drop spoonfuls of the mixture onto the parchment \\xe2\\x80\\x93 they\\xe2\\x80\\x99ll spread a little, so le

In [45]:
ingreds = ["fennel" in rec[0].lower() for rec in recipes]

In [46]:
def isIn(ingredients, find):
    for each in ingredients:
        if find in each.lower():
            return True
    return False

In [65]:
lookup = ["courgette", "onions", "peppers"]
includes_Ing = []

for rec in recipes:
    match = True
    for each in lookup:
        if not isIn(rec[2], each):
            match = False
    if match:
        includes_Ing.append(rec)
    

In [66]:
[print(rec[1]) for rec in includes_Ing]

https://www.bbcgoodfood.com/recipes/ratatouille-hotpot


[None]